In [13]:
# !pip install kfp --upgrade
# !pip install --upgrade google-cloud-storage
# !pip3 install google-cloud-aiplatform --upgrade
# !pip3 install google-cloud-pipeline-components --upgrade
# !pip install --upgrade gcloud

In [78]:
# !gcloud services enable compute.googleapis.com \
#                        containerregistry.googleapis.com \
#                        aiplatform.googleapis.com \
#                        cloudbuild.googleapis.com \
#                        cloudfunctions.googleapis.com

Operation "operations/acf.p2-287876211803-6f3ceda3-3071-434d-b46b-ac97d7eed47c" finished successfully.


In [76]:
!gcloud auth login
!gcloud projects list

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=W64za8nexwnW1JTs6PF63I0wSAkGBz&access_type=offline&code_challenge=0XeQHg13p4kVNcHdJHytJidT63WX6ktzAXyOLYHkBmg&code_challenge_method=S256


You are now logged in as [himanshusainigcp@gmail.com].
Your current project is [buoyant-algebra-355618].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


PROJECT_ID: vertextesting-355712
NAME: vertextesting
PROJECT_NUMBER: 287876211803


In [163]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=ncbIw0oAJHRyAFbjy1rY7OLSXqquIS&access_type=offline&code_challenge=p5hQKd3FxuDihJj2yeyMKMCyDoc3SulmQBkJnW_SgcY&code_challenge_method=S256


Credentials saved to file: [C:\Users\hsain\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "vertextesting-355712" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project

In [164]:
!gcloud config set project vertextesting-355712

Updated property [core/project].


In [165]:
PROJECT_ID = "vertextesting-355712"
REGION = "us-central1" #though us-central is cheaper
# through gstutil we access storage
PIPELINE_ROOT = "gs://vertextestingpipeline/pipeline_root"

In [251]:
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component)

from kfp.v2 import compiler
from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs

In [252]:
@component(
    packages_to_install=["pandas","sklearn"],
    base_image="python:3.9",
    output_component_file="getting_data.yaml"
)
def get_data(
    train_dataset : Output[Dataset],
    test_dataset : Output[Dataset]
):

    from sklearn import datasets
    import pandas as pd
    from sklearn.model_selection import train_test_split

    data_raw = datasets.load_breast_cancer()
    data = pd.DataFrame(data_raw.data, columns=data_raw.feature_names)
    data['target'] = data_raw.target

    train, test = train_test_split(data, test_size=0.3)

    train.to_csv(train_dataset.path)
    test.to_csv(test_dataset.path)


In [272]:
@component(
    packages_to_install=["pandas","sklearn","xgboost"],
    base_image="python:3.9",
    output_component_file="training.yaml"
)
def training(
    train_data : Input[Dataset],
    test_data : Input[Dataset],
    model : Output[Model],
    metrics :Output[ClassificationMetrics],
    smetrics: Output[Metrics],
    threshold_value_str : str
) -> NamedTuple("Outputs", [("accuracy", str)]):

    from xgboost import XGBClassifier
    import pandas as pd
    import json

    data = pd.read_csv(train_data.path)

    model_xg = XGBClassifier(
        objective="binary:logistic"
    )
    model_xg.fit(
        data.drop(columns=["target"]),
        data.target,
    )

    score = model_xg.score(
        data.drop(columns=["target"]),
        data.target,
    )

    model.metadata["train_score"] = float(score)
    model.metadata["framework"] = "XGBoost"

    model_xg.save_model(model.path+".bst")

    from sklearn.metrics import roc_curve, precision_score, recall_score, accuracy_score, confusion_matrix
    
    data_test = pd.read_csv(test_data.path)

    y_scores =  model_xg.predict_proba(data_test.drop(columns=["target"]))[:, 1]
    fpr, tpr, thresholds = roc_curve(
         y_true=data_test.target.to_numpy(), y_score=y_scores, pos_label=True
    )
    metrics.log_roc_curve(fpr.tolist(), tpr.tolist(), thresholds.tolist())
    
    y_pred = model_xg.predict(data_test.drop(columns=["target"]))
    
    metrics.log_confusion_matrix(
       ["False", "True"],
       confusion_matrix(
           data_test.target, y_pred
       ).tolist(),  # .tolist() to convert np array to list.
    )

    accuracy = accuracy_score(data_test['target'],y_pred)
    smetrics.log_metric("Accuracy",accuracy)

    def threshold_check(val1, val2):
        cond = "false"
        if val1 >= val2 :
            cond = "true"
        return cond
        
    threshold_value = json.loads(threshold_value_str)
    deploy = threshold_check(accuracy,float(threshold_value["accuracy"]))

    return(str(deploy),)

In [289]:
from statistics import mode


@component(
    packages_to_install=["google-cloud-aiplatform", "sklearn"],
    base_image="python:3.9",
    output_component_file="deploy.yaml"
)
def deploy_model(
    model : Input[Model],
    project: str,
    region: str,
    vertex_endpoint: Output[Artifact],
    vertex_model: Output[Model],
):
    from google.cloud import aiplatform
    aiplatform.init(project=project, location=region)
    
    ENDPOINT_NAME = "cancerModel"
    
    #Uploading model..
    deployed_model = aiplatform.Model.upload(
        display_name="XGBoost_Model",
        artifact_uri = model.uri.replace("model",""),
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-5:latest",
        serving_container_health_route=f"/v1/models/cancer",
        description = "1st version - Cancer",
        version_description="Cancer xgboost model",
    )

    def create_endpoint():
        endpoints = aiplatform.Endpoint.list(
        filter='display_name="{}"'.format(ENDPOINT_NAME),
        order_by='create_time desc',
        project=project, 
        location=region,
        )
        if len(endpoints) > 0:
            endpoint = endpoints[0]  # most recently created
        else:
            endpoint = aiplatform.Endpoint.create(
            display_name=ENDPOINT_NAME, project=project, location=region
        )
        return endpoint

    main_endpoint = create_endpoint()

    deployed_model_id = main_endpoint.list_models()
    
    if deployed_model_id is not None:
        for deployed_model in deployed_model_id:
            main_endpoint.undeploy(deployed_model_id=deployed_model.id)


    endpoint_deploy = deployed_model.deploy(machine_type="n1-standard-2", endpoint=main_endpoint, min_replica_count=1, max_replica_count=2, deployed_model_display_name="Cancer model one")
    # endpoint_deploy.wait()

    vertex_endpoint.uri = main_endpoint.resource_name
    vertex_model.uri = deployed_model.resource_name

    deployed_model = aiplatform.Model.upload(
        display_name="XGBoost_Model",
        artifact_uri = model.uri.replace("model",""),
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-5:latest",
        serving_container_health_route=f"/v1/models/cancer",
        description = "2nd version - Cancer",
        version_description="Cancer xgboost model",
    )

    print("Model ID.............")
    print(main_endpoint.list_models()[0].id)

    deployed_model_id = main_endpoint.list_models()[0].id

    endpoint_deploy = deployed_model.deploy(machine_type="n1-standard-2", endpoint=main_endpoint, min_replica_count=1, max_replica_count=1, deployed_model_display_name="Cancer model two", traffic_split={"0":70,deployed_model_id:30})
    # endpoint_deploy.wait()

    


In [290]:
@dsl.pipeline(
    pipeline_root=PIPELINE_ROOT,
    name="getting-data",
    description="Getting data of breast cancer",
)
def pipeline(
    project: str = PROJECT_ID,
    region: str = REGION
):
    pulled_data = get_data().set_cpu_limit("1").set_memory_limit("2")
    train_eval = training(train_data=pulled_data.outputs["train_dataset"], test_data=pulled_data.outputs["test_dataset"], threshold_value_str={"accuracy":0.90}).set_cpu_limit("1").set_memory_limit("2").after(pulled_data)
    
    with dsl.Condition(
        train_eval.outputs["accuracy"] == "true",
        name="Testing Threshold"
    ):
        upload_model = deploy_model(model=train_eval.outputs["model"],project=project, region=region).set_cpu_limit("1").set_memory_limit("2")

In [291]:
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='xgb_pipe.json')

c:\Users\hsain\.conda\envs\vertexai\lib\site-packages\kfp\v2\compiler\compiler.py:1278: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


In [292]:
start_pipeline = pipeline_jobs.PipelineJob(
    display_name="pipeline-breast-cancer",
    template_path="xgb_pipe.json",
    pipeline_root=PIPELINE_ROOT,
    project=PROJECT_ID,
    enable_caching=True,
    location=REGION,
)

start_pipeline.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/287876211803/locations/us-central1/pipelineJobs/getting-data-20220709190410
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/287876211803/locations/us-central1/pipelineJobs/getting-data-20220709190410')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/getting-data-20220709190410?project=287876211803


In [310]:
!pipreqs --force

INFO: Successfully saved requirements file in e:\Practicing Vertex AI\requirements.txt


In [327]:
endpoint_id = "2267236956939223040"
location = "us-central1"

In [335]:
#sample input list for prediction data
from traceback import print_tb


instance = [[14.6,23.29,93.97]]

PROJECT_ID = "vertextesting-355712"

def endpoint_predict(
    project: str, location: str, instances: list, endpoint: str
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint)

    prediction = endpoint.predict(instances=instances)
    
    return prediction

endpoint_predict(PROJECT_ID, location, instance, endpoint_id)

Prediction(predictions=[0.8395712375640869], deployed_model_id='3088558948748361728', model_version_id='', model_resource_name='projects/287876211803/locations/us-central1/models/6403076333097713664', explanations=None)